<a href="https://colab.research.google.com/github/DJCordhose/ml-resources/blob/main/notebooks/cv/faces-cnn-export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exporting model to tensorflow.js format

https://www.tensorflow.org/js/tutorials/conversion/import_keras

In [22]:
import sys

if 'google.colab' in sys.modules:
  !pip install -q tensorflowjs

In [41]:
from google.colab import drive

drive_folder = "/content/drive/MyDrive/public"
model_name = 'simple-vgg.h5'
# model_name = 'resent50.h5'
# export_dir = drive_folder + "/tfjs-model-resent50"
export_dir = drive_folder + "/tfjs-model-simple-vgg"

if 'google.colab' in sys.modules:
  drive.mount("/content/drive")
  !cp {drive_folder}/model/{model_name} .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
!ls -l

total 303088
drwx------ 6 root root      4096 Oct 26 21:35 drive
-rw-r--r-- 1 root root   4194304 Oct 26 21:39 group1-shard1of2.bin
-rw-r--r-- 1 root root   2446356 Oct 26 21:39 group1-shard2of2.bin
-rw-r--r-- 1 root root      7080 Oct 26 21:39 model.json
-rw------- 1 root root 283704376 Oct 26 22:15 resent50.h5
drwxr-xr-x 1 root root      4096 Oct 25 13:35 sample_data
-rw------- 1 root root  19990344 Oct 26 22:45 simple-vgg.h5


In [43]:
import tensorflowjs as tfjs
import tensorflow as tf
tf.__version__, tfjs.__version__

('2.10.0', '4.0.0')

In [46]:
# option 1: load model and save it in tfjs format, will give us the current tf version
model = tf.keras.models.load_model(model_name)
tfjs.converters.save_keras_model(model, export_dir)

In [47]:
# option 2: convert directly, will stick to the tf version used for saving
# !tensorflowjs_converter --input_format keras \
#                        {model_name} \
#                        {export_dir}

In [48]:
!ls -l {export_dir}

total 6493
-rw------- 1 root root 4194304 Oct 26 22:46 group1-shard1of2.bin
-rw------- 1 root root 2446356 Oct 26 22:46 group1-shard2of2.bin
-rw------- 1 root root    7080 Oct 26 22:46 model.json


In [49]:
!head {export_dir}/model.json

{"format": "layers-model", "generatedBy": "keras v2.10.0", "convertedBy": "TensorFlow.js Converter v4.0.0", "modelTopology": {"keras_version": "2.10.0", "backend": "tensorflow", "model_config": {"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 48, 48, 1], "dtype": "float32", "sparse": false, "ragged": false, "name": "input"}}, {"class_name": "Conv2D", "config": {"name": "conv2d", "trainable": true, "dtype": "float32", "filters": 32, "kernel_size": [3, 3], "strides": [1, 1], "padding": "valid", "data_format": "channels_last", "dilation_rate": [1, 1], "groups": 1, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "MaxPoo